Training Neural Network
=====

### Reference : http://www.hanbit.co.kr/store/books/look.php?p_code=B8475831198
--------

## 1. Loss function
- 신경망의 성능을 측정하는 지표로, 현재의 신경망이 훈련 데이터를 잘 처리하는지 못하는지를  나타냄

### 1.1 Mean Squared Error

#### $E = \frac 1 2 \sum_{k}(y_k-t_k)^2$
- $y_k$ : 신경망의 출력(예측값)
- $t_k$ : 정답 레이블
- $k$ : 데이터의 차원 수

In [1]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

t = np.array([0,0,1,0,0,0,0,0,0])
y = np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0])

print(mean_squared_error(y, t))

0.09750000000000003


### 1.2 Cross Entropy Error

#### $E = -\sum t_klogy_k$

### $E = - \frac 1 N \sum_{n} \sum_{k} t_{nk} logy_{nk}$

- 단일 데이터가 아닌 N개의 데이터에 대한 cross entropy error
- k개의 데이터를 임의로 추출하여 사용하는 mini-batch 방법을 사용함
- $t_{nk}$ : n번째 데이터의 k번째 값
- $y_{nk}$ : n번째 데이터의 k번째 예측값
- 마지막에 데이터의 크기 N으로 나눠줌으로써 정규화해준다. $\rightarrow$ 평균 손실 함수

In [2]:
def cross_entropy_error(y, t):
    delta = 1e-7  # inf가 되는 경우를 방지하기 위해 작은 상수를 더해줌
    return -np.sum(t * np.log(y + delta))

print(cross_entropy_error(y, t))

0.510825457099338


### 2. Mini Batch

In [3]:
from keras.datasets.mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data()
x_train = x_train.reshape((60000, 784))

# 임의의 훈련 데이터 추출(mini-batch)
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

print(batch_mask)

Using TensorFlow backend.


[16822 35436  8293 38243 26890  8686 47950 32166  3477  4594]


In [4]:
# 배치용 교차 엔트로피 오차 구현
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * log(y + 1e-7)) / batch_size

In [5]:
# label이 one-host encoding되지 않은 경우
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### 3. 수치 미분

In [6]:
def numerical_diff(f, x):
    h = 1e-4  # 너무 작은 값은 반올림 오차로 표기가 안됨
    return (f(x+h) - f(x-h)) / (2**h)  # x+h와 x값 사이의 차분에서 오차가 생김. 중심 차분을 사용!

#### 3.1 수치 미분의 예

In [7]:
import matplotlib.pyplot as plt

def function_1(x):
    return 0.01 * x ** 2 + 0.1 * x

x = np.arange(0.0, 20.0, 0.1)
y = function_1(x)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.plot(x, y)
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
print(numerical_diff(function_1, 5))
print(numerical_diff(function_1, 10))

3.9997227507184116e-05
5.9995841260776174e-05


#### 3.2 편미분

$f(x_0, x_1) = x_0^2 + x_1^2$

Q1) $x_0=3, x_1=4, x_0$에 대한 편미분  
Q2) $x_0=3, x_1=4, x_1$에 대한 편미분  

In [9]:
def function_2(x):
    return x[0] ** 2 + x[1] ** 2

# Q1
def function_tmp1(x0):
    return x0 * x0 + 4.0 ** 2.0

print(numerical_diff(function_tmp1, 3.0))


# Q2
def function_tmp2(x1):
    return 3.0**2.0 + x1*x1

print(numerical_diff(function_tmp2, 4.0))

0.0011999168252217404
0.0015998891002944695


### 4. 기울기

- 모든 변수의 편미분을 묶어서 ${(\frac {\partial f} {\partial x_0}, \frac {\partial f} {\partial x_1})}$ 벡터 형식으로 정리한 것을 기울기

In [29]:
# 기울기 구현
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [13]:
print(numerical_gradient(function_2, np.array([3.0, 4.0])))
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


### 4.1 경사법(경사하강법)

- 함수의 기울어진 방향으로 이동하면서 함수의 극값을 찾는 방법

$x_0 = x_0 - \eta \frac {\partial f} {\partial x_0}$

$x_1 = x_1 - \eta \frac {\partial f} {\partial x_1}$

- $\eta$ : 학습률(한 번의 학습으로 얼마만큼 학습해야할지를 결정)

In [15]:
def gradient_descent(f, init_x, lr=0.01, step_num=100): # step_num : 반복횟수
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [16]:
# 경사법으로 최솟값 구하기
def funtion_2(x):
    return x[0]**2 + x[1] ** 2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [19]:
# 학습률이 너무 큰 예 lr = 10.0
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100))

# 학습률이 너무 작은 예 lr = 1e-10
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100))

[-2.58983747e+13 -1.29524862e+12]
[-2.99999994  3.99999992]


### 4.2 신경망에서의 기울기

$\frac {\partial L} {\partial w_{11}}$ : $w_{11}$을 조금 변경했을 때 손실함수 $L$이 얼만큼 변했는지를 나타냄

In [33]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss
    
net = simpleNet()
print(net.W)

[[-0.22795127 -0.20225471 -0.63957805]
 [-1.39089294  0.04005808  0.08311462]]


In [23]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

# 최댓값의 인덱스(신경망이 예측한 정답)
print(np.argmax(p))

t = np.array([0,0,1]) # 정답 레이블
net.loss(x, t)

[-1.3885744  -0.08530056 -0.30894367]
1


4.790990477931121

In [30]:
f = lambda w : net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[-0.96393121  0.26904363  0.69488758]
 [-1.44589682  0.40356545  1.04233137]]


### 5. 학습 알고리즘 구현하기

#### 1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져온다. 
#### 2. 기울기 산출 : 미니배치의 손실 함수 값을 줄이기 위해 매개변수의 기울기를 구함
#### 3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다
#### 4. 반복 : 1~3단계를 반복한다

- 데이터를 미니배치로 무작위로 선정하기 때문에 확률적 경사 하강법(Stochastic Gradient Descent)를 실행한다 

#### 5.1 2층 신경망 클래스 구현하기

In [42]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std=0.01):
        
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    # 신경망을 통한 예측값 
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # 손실함수 값
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 신경망의 예측 정확도
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    # 기울기 구현
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [43]:
# 인스턴스 만들어보기
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [44]:
x = np.random.randn(100, 784) # 입력값
y = net.predict(x)

#### 5.2 미니배치 학습 구현하기

In [ ]:
import numpy as np
from keras.datasets.mnist import load_data

(x_train, t_train), (x_test, t_test) = load_data()
x_train = x_train.reshape((60000, 784))

train_loss_list = []

# 하이퍼파라미터
iter_nums = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iter_nums):
    
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

#### 5.3 시험 데이터로 평가하기

In [49]:
import numpy as np
from keras.datasets.mnist import load_data

(x_train, t_train), (x_test, t_test) = load_data()
x_train = x_train.reshape((60000, 784))

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 하이퍼파라미터
iter_nums = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iter_nums):
    
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산 
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print('train acc, test acc | ' + str(train_acc) + ', ' + str(test_acc))